In [ ]:
import sys
sys.path.append("..")  

import torch
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks

from src.data.dataset import MultiViewCocoDataset

def visualize_view(img, target, title="View"):
    """tv_tensors形式の画像とターゲットを描画して表示する"""
    # 1. 画像をuint8に変換（描画用）
    if img.dtype != torch.uint8:
        # Normalizeされている場合は戻す必要があるが、簡易的に[0,1]を[0,255]へ
        img_int = (img * 255).to(torch.uint8) if img.max() <= 1.0 else img.to(torch.uint8)
    else:
        img_int = img

    # 2. ボックスの描画
    boxes = target["boxes"]
    result_img = draw_bounding_boxes(img_int, boxes, colors="red", width=2)

    # 3. マスクがある場合は重ねる
    if "masks" in target:
        masks = target["masks"].bool()
        result_img = draw_segmentation_masks(result_img, masks, alpha=0.5)

    # 表示
    plt.figure(figsize=(8, 8))
    plt.imshow(result_img.permute(1, 2, 0).cpu().numpy())
    plt.title(title)
    plt.axis("off")
    plt.show()

def test_dataset():
    # 変換の定義 (前述のSSL用オーギュメンテーション)
    from your_transform_file import get_train_transforms_coco # 作成した関数をインポート
    
    transforms = get_train_transforms_coco(size=(320, 320))
    
    # 1. データセットの初期化
    # rootは適宜書き換えてください
    dataset = MultiViewCocoDataset(
        root="./data/coco", 
        split="train", 
        transforms=transforms, 
        num_crops=2
    )

    print(f"Dataset length: {len(dataset)}")

    # 2. 1件取得
    views_img, views_target = dataset[0]

    print(f"Generated {len(views_img)} views.")

    # 3. 各ビューを可視化
    for i, (img, target) in enumerate(zip(views_img, views_target)):
        print(f"View {i} - Image shape: {img.shape}, Boxes: {len(target['boxes'])}")
        visualize_view(img, target, title=f"View {i}")

if __name__ == "__main__":
    test_dataset()